In [1]:
import micom
import micom.measures
import pandas as pd
import numpy as np 
import scipy.stats
from matplotlib.patches import Rectangle

import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *

%matplotlib inline

## Load Taxonomic Data
Arivale metagenomes mapped to taxonomy with Kraken2

In [6]:
taxonomy = pd.read_csv('../data/arivale-metagenomes.csv')
taxonomy['sample_id'] = taxonomy['sample_id'].astype('str')

## Add Simulated Probiotics
Add five-species probiotic cocktail to each Arivale sample

In [7]:
def add_all_probiotic(taxonomy):
    """Adds probiotic cocktail of AMUC, CBEI, CBUT, BINF, and EHAL to existing taxonomy table at 1% RA each"""
    taxonomy_probiotic_total = pd.DataFrame()
    for sample in taxonomy['sample_id'].unique():
        taxonomy_reduced = taxonomy[taxonomy['sample_id'] == sample].copy()
        taxonomy_reduced['abundance'] = taxonomy_reduced['abundance']*0.95
        taxonomy_probiotic = pd.concat([taxonomy_reduced,pd.DataFrame({
                  'sample_id':[sample, sample, sample, sample, sample], 
                  'species': ['Akkermansia muciniphila','Clostridium beijerinckii',
                              'Clostridium butyricum','Bifidobacterium longum',
                              'Anaerobutyricum hallii'],
                  'abundance':[0.01, 0.01, 0.01, 0.01, 0.01], 
                  'id':['Akkermansia_muciniphila','Clostridium_beijerinckii',
                              'Clostridium_butyricum','Bifidobacterium_longum',
                              'Anaerobutyricum_hallii']})])
        taxonomy_probiotic_total = pd.concat([taxonomy_probiotic_total, taxonomy_probiotic])

    return taxonomy_probiotic_total
# Add probiotics
taxonomy_probiotic = add_all_probiotic(taxonomy)

# Combine duplicate taxa
taxonomy_probiotic = taxonomy_probiotic.groupby(['sample_id','id','species']).sum().reset_index()

## Build Models
Build models for all samples in the Arivale cohort

In [ ]:
agora = '../agora_models/data'
manifest_treatment = micom.workflows.build(taxonomy_probiotic,
                                 model_db=agora, 
                                 out_folder='../Arivale_treatment_models',
                                 cutoff=0.001, 
                                 threads=10)
manifest_baseline = micom.workflows.build(taxonomy,
                                 model_db=agora, 
                                 out_folder='../Arivale_baseline_models',
                                 cutoff=0.001, 
                                 threads=10)

Output()

[06/17/25 03:08:42] WARNING  Less than 50% of the abundance     ]8;id=248784;file:///users/nbohmann/miniconda3/envs/micom2024/lib/python3.9/site-packages/micom/community.py\community.py]8;;\:]8;id=389187;file:///users/nbohmann/miniconda3/envs/micom2024/lib/python3.9/site-packages/micom/community.py#229\229]8;;\
                             could be matched to the model                      
                             database. Model `22001612560204`                   
                             may not be representative of the                   
                             sample                                             
[06/17/25 03:08:42] WARNING  Less than 50% of the abundance     ]8;id=340494;file:///users/nbohmann/miniconda3/envs/micom2024/lib/python3.9/site-packages/micom/community.py\community.py]8;;\:]8;id=792257;file:///users/nbohmann/miniconda3/envs/micom2024/lib/python3.9/site-packages/micom/community.py#229\229]8;;\
                             could

## Prepare media
Load the standard European medium, and augment with addition fiber prebiotics

In [14]:
# Load European medium
eu_medium = micom.qiime_formats.load_qiime_medium('../european_medium.qza')

# Make copies for adding prebiotics
eu_medium_inulin = eu_medium.copy()
eu_medium_pectin = eu_medium.copy()
eu_medium_malto = eu_medium.copy()
eu_medium_starch = eu_medium.copy()
eu_medium_hemp = eu_medium.copy()
eu_medium_psyllium= eu_medium.copy()

# Add prebiotics
eu_medium_inulin.loc['EX_inulin_m', 'flux'] = 6.14
eu_medium_pectin.loc['EX_pect_m', 'flux'] = 0.4
eu_medium_starch.loc['EX_strch1_m', 'flux'] = 16.65
eu_medium_malto.loc['EX_dextrin_m', 'flux'] = 30.28
eu_medium_hemp.loc['EX_cellul_m', 'flux'] = 0.37
eu_medium_psyllium.loc['EX_arabinoxyl_m', 'flux'] = 4.09

# Load high fiber medium
hf_medium = micom.qiime_formats.load_qiime_medium('../high_fiber_medium.qza')

## Grow samples
Grow samples for both probiotic treated and untreated samples using each medium. This will take a while!

In [15]:
manifest_treatment = pd.read_csv('../Arivale_treament_models/manifest.csv')
treatment_noPrebiotic = micom.workflows.grow(manifest_treatment, 
                                  model_folder = '../Arivale_treament_models/',
                                  medium = eu_medium,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
treatment_inulin = micom.workflows.grow(manifest_treatment, 
                                  model_folder = '../Arivale_treament_models/',
                                  medium = eu_medium_inulin,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
treatment_pect = micom.workflows.grow(manifest_treatment, 
                                  model_folder = '../Arivale_treament_models/',
                                  medium = eu_medium_pectin,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
treatment_starch = micom.workflows.grow(manifest_treatment, 
                                  model_folder = '../Arivale_treament_models/',
                                  medium = eu_medium_starch,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
treatment_malto = micom.workflows.grow(manifest_treatment, 
                                  model_folder = '../Arivale_treament_models/',
                                  medium = eu_medium_malto,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
treatment_hemp = micom.workflows.grow(manifest_treatment, 
                                  model_folder = '../Arivale_treament_models/',
                                  medium = eu_medium_hemp,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
treatment_psyllium = micom.workflows.grow(manifest_treatment, 
                                  model_folder = '../Arivale_treament_models/',
                                  medium = eu_medium_psyllium,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
treatment_hf = micom.workflows.grow(manifest_treatment, 
                                  model_folder = '../Arivale_treament_models/',
                                  medium = hf_medium,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)


manifest_baseline = pd.read_csv('../Arivale_baseline_models/manifest.csv')
baseline_noPrebiotic = micom.workflows.grow(manifest_baseline, 
                                  model_folder = '../Arivale_baseline_models/',
                                  medium = eu_medium,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
baseline_inulin = micom.workflows.grow(manifest_baseline, 
                                  model_folder = '../Arivale_baseline_models/',
                                  medium = eu_medium_inulin,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
baseline_pect = micom.workflows.grow(manifest_baseline, 
                                  model_folder = '../Arivale_baseline_models/',
                                  medium = eu_medium_pectin,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
baseline_starch = micom.workflows.grow(manifest_baseline, 
                                  model_folder = '../Arivale_baseline_models/',
                                  medium = eu_medium_starch,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
baseline_malto = micom.workflows.grow(manifest_baseline, 
                                  model_folder = '../Arivale_baseline_models/',
                                  medium = eu_medium_malto,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
baseline_hemp = micom.workflows.grow(manifest_baseline, 
                                  model_folder = '../Arivale_baseline_models/',
                                  medium = eu_medium_hemp,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
baseline_psyllium = micom.workflows.grow(manifest_baseline, 
                                  model_folder = '../Arivale_baseline_models/',
                                  medium = eu_medium_psyllium,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)
baseline_hf = micom.workflows.grow(manifest_baseline, 
                                  model_folder = '../Arivale_baseline_models/',
                                  medium = hf_medium,
                                  tradeoff = 0.99, 
                                  strategy = 'none',                                    
                                  threads = 20)

FileNotFoundError: [Errno 2] No such file or directory: '../Arivale_treament_models/manifest.csv'

## Collect Growth Rates
Return the MCMM-predicted growth rates for each prebiotic/probiotic context

In [ ]:
# Gather growth rates from treatment samples
rates_noPrebiotic = treatment_noPrebiotic.growth_rates
rates_noPrebiotic['treatment'] = 'No Prebiotic'
rates_noPrebiotic['probiotic'] = '+'
rates_inulin = treatment_inulin.growth_rates
rates_inulin['treatment'] = 'Inulin'
rates_inulin['probiotic'] = '+'
rates_pect = treatment_pect.growth_rates
rates_pect['treatment'] = 'Pectin'
rates_pect['probiotic'] = '+'
rates_starch = treatment_starch.growth_rates
rates_starch['treatment'] = 'Starch'
rates_starch['probiotic'] = '+'
rates_malto = treatment_malto.growth_rates
rates_malto['treatment'] = 'Maltodextrin'
rates_malto['probiotic'] = '+'
rates_hemp = treatment_hemp.growth_rates
rates_hemp['treatment'] = 'Hemp Seed'
rates_hemp['probiotic'] = '+'
rates_psyllium = treatment_psyllium.growth_rates
rates_psyllium['treatment'] = 'Psyllium Husk'
rates_psyllium['probiotic'] = '+'
rates_hf = treatment_hf.growth_rates
rates_hf['treatment'] = 'High Fiber'
rates_hf['probiotic'] = '+'

# Baseline samples
baseline_rates_noPrebiotic = baseline_noPrebiotic.growth_rates
baseline_rates_noPrebiotic['treatment'] = 'No Prebiotic'
baseline_rates_noPrebiotic['probiotic'] = '-'
baseline_rates_inulin = baseline_inulin.growth_rates
baseline_rates_inulin['treatment'] = 'Inulin'
baseline_rates_inulin['probiotic'] = '-'
baseline_rates_pect = baseline_pect.growth_rates
baseline_rates_pect['treatment'] = 'Pectin'
baseline_rates_pect['probiotic'] = '-'
baseline_rates_starch = baseline_starch.growth_rates
baseline_rates_starch['treatment'] = 'Starch'
baseline_rates_starch['probiotic'] = '-'
baseline_rates_malto = baseline_malto.growth_rates
baseline_rates_malto['treatment'] = 'Maltodextrin'
baseline_rates_malto['probiotic'] = '-'
baseline_rates_hemp = baseline_hemp.growth_rates
baseline_rates_hemp['treatment'] = 'Hemp Seed'
baseline_rates_hemp['probiotic'] = '-'
baseline_rates_psyllium = baseline_pysllium.growth_rates
baseline_rates_psyllium['treatment'] = 'Psyllium Husk'
baseline_rates_psyllium['probiotic'] = '-'
baseline_rates_hf = baseline_hf.growth_rates
baseline_rates_hf['treatment'] = 'High Fiber'
baseline_rates_hf['probiotic'] = '-'


# Concatenate growth rates into a single DF
rates_probiotic = pd.concat([rates_noPrebiotic, rates_inulin, rates_pect, rates_starch, rates_malto, rates_hemp, rates_psyllium, rates_hf,
                            baseline_rates_noPrebiotic, baseline_rates_inulin, baseline_rates_pect, baseline_rates_starch, baseline_rates_malto,baseline_rates_hemp,baseline_rates_psyllium,
                             baseline_rates_hf])



# Isolate Probiotics
rates_probiotic = rates_probiotic[rates_probiotic.index.isin(['Akkermansia_muciniphila',
                                                              'Clostridium_beijerinckii',
                                                              'Clostridium_butyricum',
                                                              'Bifidobacterium_longum',
                                                              'Anaerobutyricum_hallii',
                                                              ])].reset_index()


# Abbreviate probiotics for clarity
rates_probiotic.compartments = rates_probiotic.compartments.str.replace('Akkermansia_muciniphila','AMUC').str.replace(
                                                          'Clostridium_beijerinckii','CBEI').str.replace(
                                                          'Clostridium_butyricum','CBUT').str.replace(
                                                          'Bifidobacterium_longum','BINF').str.replace(
                                                          'Anaerobutyricum_hallii', 'AHAL')

# Create unique sample ID for each treatment combination
rates_probiotic['sample_id'] = rates_probiotic['sample_id'].astype('str')+'_'+rates_probiotic['treatment']+'_'+rates_probiotic['probiotic']

# Log scale growth rates
rates_probiotic['growth_rate_scaled'] = np.log10(rates_probiotic[['growth_rate']]+0.00000001)

# Threshold log-scaled growth rates
rates_probiotic.loc[rates_probiotic['growth_rate_scaled']>=-3, 'MCMM_engraftment'] = 1
rates_probiotic['MCMM_engraftment'].fillna(0.0, inplace = True)

# Pivot data so rows are samples and columns and probiotic species
rates_probiotic_pvt = pd.pivot_table(rates_probiotic, 
                    columns='compartments',
                    index='sample_id',
                    values='MCMM_engraftment').fillna(0.0)

# Format for plotting
rates_probiotic_pvt['treatment'] = rates_probiotic_pvt.index.str.split('_').str[1]
rates_probiotic_pvt['probiotic'] = rates_probiotic_pvt.index.str.split('_').str[2]
rates_probiotic_pvt = rates_probiotic_pvt.sort_values(by = 'treatment')

# Order and set row colors
custom_order = ['No Prebiotic','Inulin','Pectin','Starch','Maltodextrin','Hemp Seed','Psyllium Husk','High Fiber']
rates_probiotic_pvt['treatment'] = pd.Categorical(rates_probiotic_pvt['treatment'], categories=custom_order, ordered=True)
rates_probiotic_pvt = rates_probiotic_pvt.sort_values('treatment')

palette = sns.color_palette("Set2", 8)
sns.set(font_scale=3)
lut = dict(zip(rates_probiotic_pvt['treatment'].unique(),palette))
row_colors = rates_probiotic_pvt['treatment'].map(lut)

# Save output
rates_probiotic.to_csv('../arivale_probiotic_rates.csv')

## Plot Engraftment of Probiotics

In [ ]:
x = sns.clustermap(rates_probiotic_pvt[rates_probiotic_pvt['probiotic'] == '+'].iloc[:,:-2].T, yticklabels=True, 
                   col_colors = row_colors, col_cluster = False, xticklabels=False,  cmap='gray_r',cbar_pos=None,   # <- This removes the color bar
                   figsize=(30, 8), dendrogram_ratio=0.05)

for label in rates_probiotic_pvt['treatment'].unique():
    x.ax_col_dendrogram.bar(0, 0, color=lut[label],
                            label=label, linewidth=0)
x.ax_col_dendrogram.legend(loc='upper center', bbox_to_anchor=(0.5, 6), ncol=4)
x.savefig('../figures/arivale_heatmap.svg', dpi=200)


## Calculate Production Rates
Return the MCMM-predicted metabolic production for each prebiotic/probiotic combination

In [ ]:
# Gather production rates from treatment samples
production_noPrebiotic = micom.measures.production_rates(treatment_noPrebiotic)
production_noPrebiotic['treatment'] = 'No Prebiotic'
production_noPrebiotic['probiotic'] = '+'

production_inulin = micom.measures.production_rates(treatment_inulin)
production_inulin['treatment'] = 'Inulin'
production_inulin['probiotic'] = '+'

production_pect = micom.measures.production_rates(treatment_pect)
production_pect['treatment'] = 'Pectin'
production_pect['probiotic'] = '+'

production_starch = micom.measures.production_rates(treatment_starch)
production_starch['treatment'] = 'Starch'
production_starch['probiotic'] = '+'

production_malto = micom.measures.production_rates(treatment_malto)
production_malto['treatment'] = 'Maltodextrin'
production_malto['probiotic'] = '+'

production_hemp = micom.measures.production_rates(treatment_hemp)
production_hemp['treatment'] = 'Hemp Seed'
production_hemp['probiotic'] = '+'

production_psyllium = micom.measures.production_rates(treatment_psyllium)
production_psyllium['treatment'] = 'Psyllium Husk'
production_psyllium['probiotic'] = '+'

production_hf = micom.measures.production_rates(treatment_hf)
production_hf['treatment'] = 'High Fiber'
production_hf['probiotic'] = '+'

# Baseline samples
baseline_production_noPrebiotic = micom.measures.production_rates(baseline_noPrebiotic)
baseline_production_noPrebiotic['treatment'] = 'No Prebiotic'
baseline_production_noPrebiotic'probiotic'] = '-'

baseline_production_inulin = micom.measures.production_rates(baseline_inulin)
baseline_production_inulin['treatment'] = 'Inulin'
baseline_production_inulin['probiotic'] = '-'

baseline_production_pect = micom.measures.production_rates(baseline_pect)
baseline_production_pect['treatment'] = 'Pectin'
baseline_production_pect['probiotic'] = '-'

baseline_production_starch = micom.measures.production_rates(baseline_starch)
baseline_production_starch['treatment'] = 'Starch'
baseline_production_starch['probiotic'] = '-'

baseline_production_malto = micom.measures.production_rates(baseline_malto)
baseline_production_malto['treatment'] = 'Maltodextrin'
baseline_production_malto['probiotic'] = '-'

baseline_production_hemp = micom.measures.production_rates(baseline_hemp)
baseline_production_hemp['treatment'] = 'Hemp Seed'
baseline_production_hemp['probiotic'] = '-'

baseline_production_psyllium = micom.measures.production_rates(baseline_psyllium)
baseline_production_psyllium['treatment'] = 'Psyllium Husk'
baseline_production_psyllium['probiotic'] = '-'


baseline_production_hf = micom.measures.production_rates(baseline_hf)
baseline_production_hf['treatment'] = 'High Fiber'
baseline_production_hf['probiotic'] = '-'

# Concatenate results
production = pd.concat([production_baseline, production_inulin, production_pect, production_starch, production_malto, production_hemp, production_psyllium, production_hf,
                       baseline_production_baseline, baseline_production_inulin, baseline_production_pect, baseline_production_starch, baseline_production_malto,
                        baseline_production_hemp, baseline_production_psyllium, baseline_production_hf])

# Isolate butyrate and propionate
production_but = production[production['metabolite'] == 'but[e]']
production_ppa = production[production['metabolite'] == 'ppa[e]']
production_but

production.to_csv('../arivale_metabolic_production.csv')

## Plot Butyrate Production
Boxplots display butyrate production between prebiotic and probiotic contexts

In [16]:
arivale_butyrate = (ggplot(production_but, aes(x = 'treatment', y = 'flux'))
                    +geom_boxplot(aes(fill = 'probiotic'))
                    +scale_fill_manual(values = {'+': '#5B85AA',
                                                 '-': '#F83860'
                                                })
                    +scale_x_discrete(limits = ['No Prebiotic', 'Inulin','Pectin','Starch','Maltodextrin','Psyllium Husk','Hemp Seed','High Fiber'])
                    +labs(y = 'Predicted Butyate \n mmol/gDW/h', x = '',title = 'Butyrate', fill = 'Probiotic Treatment')
                    +theme_minimal()    
                    +ylim(0, 100)
                    +theme(figure_size=(8, 8),legend_position='none', text=element_text(size=24),axis_text_x=element_text(rotation=33, hjust=1))
                   )


ggsave(arivale_butyrate, '../figures/arivale_butyrate.svg', dpi = 300)

arivale_butyrate

NameError: name 'production_but' is not defined

## Test Significance
Use non-parametric Mann-Whitney to determine signficance in difference of propionate production between probiotic and baseline conditions

In [17]:
scipy.stats.mannwhitneyu(production_but[(production_but['treatment'] == 'No Prebiotic')&(production_but['probiotic']== '+')]['flux'],
production_but[(production_but['treatment'] == 'No Prebiotic')&(production_but['probiotic']== '-')]['flux'])

NameError: name 'production_but' is not defined

## Plot Propionate Production
Boxplots display propionate production between prebiotic and probiotic contexts

In [18]:
arivale_propionate = (ggplot(production_ppa, aes(x = 'treatment', y = 'flux'))
                    +geom_boxplot(aes(fill = 'probiotic'))
                    +theme_seaborn()
                    +scale_fill_manual(values = {'+': '#5B85AA',
                                                 '-': '#F83860'
                                                })
                    +scale_x_discrete(limits = ['No Prebiotic', 'Inulin','Pectin','Starch','Maltodextrin','Psyllium Husk','Hemp Seed','High Fiber'])
                    +labs(y = 'Predicted Propionate \n mmol/gDW/h', x = '',title = 'Propionate', fill = 'Probiotic Treatment')
                    +theme_minimal()    
                    +ylim(0, 400)
                    +theme(figure_size=(12, 8), text=element_text(size=24),axis_text_x=element_text(rotation=33, hjust=1))
                   )


ggsave(arivale_propionate, '../figures/arivale_propionate.svg', dpi = 300)

arivale_propionate

NameError: name 'production_ppa' is not defined

## Test Significance
Use non-parametric Mann-Whitney to determine signficance in difference of propionate production between conditions

In [ ]:
scipy.stats.mannwhitneyu(production_ppa[(production_ppa['treatment'] == 'No Prebiotic')&(production_ppa['probiotic']== '+')]['flux'],
production_ppa[(production_ppa['treatment'] == 'No Prebiotic')&(production_ppa['probiotic']== '-')]['flux'])

## Format DF
Reformat these data to allow us to find the most optimal intervention for each individual

In [ ]:
# Create unique ID for prebiotic/probiotic combos
production_but['combined_treatment'] = production_but['treatment']+'_'+production_but['probiotic']
production_ppa['combined_treatment'] = production_ppa['treatment']+'_'+production_ppa['probiotic']

# Pivot so each row is a treatment, and each column is a sample
production_but_pvt = pd.pivot_table(production_but, 
                                    columns = 'sample_id',
                                    index = 'combined_treatment', 
                                    values = 'flux')
production_ppa_pvt = pd.pivot_table(production_ppa, 
                                    columns = 'sample_id',
                                    index = 'combined_treatment', 
                                    values = 'flux')

# Remove high-fiber results
production_but_pvt_noHF = production_but_pvt[~production_but_pvt.index.str.contains('High Fiber')]
production_ppa_pvt_noHF = production_ppa_pvt[~production_ppa_pvt.index.str.contains('High Fiber')]

# Find Optimal Intervention
Isolate intervention that leads to maximal butyrate/propionate production in each individual

In [ ]:
# Create optimal propionate DF
optimal_ppa = pd.DataFrame({'Optimal':production_ppa_pvt_noHF.max(),
                            'Standard':production_ppa_pvt.loc['Inulin_+']}).reset_index()

# Create optimal butyrate DF
optimal_but = pd.DataFrame({'Optimal':production_but_pvt_noHF.max(),
                            'Standard':production_but_pvt.loc['Inulin_+']}).reset_index()

# Melt DFs 
optimal_ppa_melt = pd.melt(optimal_ppa, 
                id_vars = 'sample_id', 
                value_vars = ['Optimal','Standard'], 
                value_name = 'flux', 
                var_name = 'type')

optimal_but_melt = pd.melt(optimal_but, 
                id_vars = 'sample_id', 
                value_vars = ['Optimal','Standard'], 
                value_name = 'flux', 
                var_name = 'type')

## Plot Optimal Propionate

In [ ]:
optimal_plot = (ggplot(optimal_ppa_melt, aes(x = 'type', y = 'flux'))
                    +geom_boxplot(aes(fill = 'type'))
                    +scale_fill_manual(values = {'Optimal': '#b2d0b1',
                                                 'Standard': '#fee3b2'
                                                })
                    +labs(y = 'Predicted Propionate , mmol/gDW/h', x = '',title = 'Propionate', fill = 'Probiotic Treatment')
                    +theme_minimal()    
                    +theme(figure_size=(10, 5),
                           legend_position='none',
                           axis_title=element_text(size=18),
                            axis_text=element_text(size=18),
                            plot_title=element_text(size=20)))
                
               
ggsave(optimal_plot, '../figures/optimal_propionate.svg',dpi =300)
optimal_plot

## Plot Optimal Butyrate

In [ ]:
optimal_plot = (ggplot(optimal_but_melt, aes(x = 'type', y = 'flux'))
                    +geom_boxplot(aes(fill = 'type'))
                    +scale_fill_manual(values = {'Optimal': '#b2d0b1',
                                                 'Standard': '#fee3b2'
                                                })
                    +labs(y = 'Predicted Butyrate, mmol/gDW/h', x = '',title = 'Butyrate', fill = 'Probiotic Treatment')
                    +theme_minimal()    
                    +theme(figure_size=(10, 5),
                           legend_position='none',
                           axis_title=element_text(size=18),
        axis_text=element_text(size=18),
        plot_title=element_text(size=20))               )
ggsave(optimal_plot, '../figures/optimal_butyrate.svg',dpi =300)
optimal_plot



## Reformat Indicies for Supplementary Figures

In [ ]:
production_ppa_pvt_noHF.index = production_ppa_pvt_noHF.index.str.replace('_+', ', Probiotic')
production_ppa_pvt_noHF.index = production_ppa_pvt_noHF.index.str.replace('_-', ', No Probiotic')
production_ppa_pvt_noHF = production_ppa_pvt_noHF.T

production_but_pvt_noHF.index = production_but_pvt_noHF.index.str.replace('_+', ', Probiotic')
production_but_pvt_noHF.index = production_but_pvt_noHF.index.str.replace('_-', ', No Probiotic')
production_but_pvt_noHF = production_but_pvt_noHF.T

## Plot Personalized Prediction Heatmaps 

In [ ]:
max_indicies = production_but_pvt_noHF.idxmax(axis = 1)


ax = sns.clustermap(production_but_pvt_noHF.T,
                    cmap = 'Reds',
                    figsize = (20,15),
                    fmt='',
                    xticklabels = False,
                    col_colors = row_colors, 
                    col_cluster = False, 
                    row_cluster = False,
                    cbar_pos= [0.05,0.05,.05,.72],
                    annot_kws={'fontsize': 18, 'color':'white','verticalalignment': 'center'})

for x in (max_indicies.index):
    ax.ax_heatmap.add_patch(Rectangle((production_but_pvt_noHF.index.get_loc(x),
                                      production_but_pvt_noHF.columns.get_loc(max_indicies[x])),
                                       1, 1, fill=False, edgecolor='black', lw=3))


ax.savefig("personalized_but.svg", dpi=300)


In [ ]:
max_indicies = production_ppa_pvt_noHF.idxmax(axis = 1)


ax = sns.clustermap(production_ppa_pvt_noHF.T,
                    cmap = 'Reds',
                    figsize = (20,15),
                    fmt='',
                    xticklabels = False,
                    col_colors = row_colors, 
                    col_cluster = False, 
                    row_cluster = False,
                    cbar_pos= [0.05,0.05,.05,.72],
                    annot_kws={'fontsize': 18, 'color':'white','verticalalignment': 'center'})

for x in (max_indicies.index):
    ax.ax_heatmap.add_patch(Rectangle((production_ppa_pvt_noHF.index.get_loc(x),
                                      production_ppa_pvt_noHF.columns.get_loc(max_indicies[x])),
                                       1, 1, fill=False, edgecolor='black', lw=3))

ax.savefig("personalized_ppa.svg", dpi=300)
